In [1]:
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'

In [3]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [4]:
train=pd.DataFrame()
train['image'], train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test=pd.DataFrame()
test['image'], test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [8]:
from tqdm.auto import tqdm


In [9]:
!pip install --upgrade jupyter jupyterlab ipywidgets

  Using cached jupyterlab-4.4.0-py3-none-any.whl.metadata (16 kB)


In [10]:
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,color_mode='grayscale')
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features

In [11]:
!pip install ipywidgets

In [12]:
!pip install --upgrade ipywidgets

In [13]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [14]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [15]:
x_train=train_features/255.0
x_test=test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [19]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [20]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))


E:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])  # ✅ Fix: metrics should be a list


In [23]:
model.fit(x=x_train,y=y_train,batch_size=128, epochs=100, validation_data=(x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 235s 995ms/step - accuracy: 0.2326 - loss: 1.8392 - val_accuracy: 0.2584 - val_loss: 1.8029
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 216s 954ms/step - accuracy: 0.2468 - loss: 1.8039 - val_accuracy: 0.3014 - val_loss: 1.7096
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 211s 934ms/step - accuracy: 0.3048 - loss: 1.6990 - val_accuracy: 0.4036 - val_loss: 1.5443
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 212s 939ms/step - accuracy: 0.3874 - loss: 1.5537 - val_accuracy: 0.4524 - val_loss: 1.3960
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 225s 994ms/step - accuracy: 0.4332 - loss: 1.4595 - val_accuracy: 0.4918 - val_loss: 1.3176
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 225s 995ms/step - accuracy: 0.4534 - loss: 1.4118 - val_accuracy: 0.5003 - val_loss: 1.2991
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 245s 1s/step - accuracy: 0.4652 - loss: 1.3715 - val_accuracy: 0.5248 - val_loss: 1.2395
Epoch 8/100
 91/226 ━━━━━━━━━━━━━━━━━━━━ 2:03 915ms/step - accuracy: 0.4

KeyboardInterrupt: 

In [24]:
model_json=model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [25]:
from keras.models import model_from_json

In [36]:
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Register 'Sequential' manually to avoid deserialization error
from tensorflow.keras.saving import register_keras_serializable
register_keras_serializable()(Sequential)

# Load model JSON
with open("facialemotionmodel.json", "r") as json_file:
    model_json = json_file.read()

# Deserialize model with fallback custom_objects
model = model_from_json(model_json, custom_objects={'Sequential': Sequential})

# Load weights
model.load_weights("facialemotionmodel.h5")

print("✅ Model loaded successfully.")


✅ Model loaded successfully.


In [35]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [37]:
label=['angry','disgust','fear','happy','neutral','sad','surprise']

In [38]:
def ef(image):
    img=load_img(img,grayscale=True)
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [41]:
import numpy as np

# Create a dummy grayscale image of shape (1, 48, 48, 1)
dummy_input = np.random.rand(1, 48, 48, 1)

# Predict
pred = model.predict(dummy_input)
print("Predicted output (random image):", pred)
print("Predicted class index:", pred.argmax())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
Predicted output (random image): [[0.07602965 0.00449032 0.25107047 0.04455621 0.24781223 0.3507054
  0.02533569]]
Predicted class index: 5
